In [19]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import shapely
from shapely.geometry import mapping
# from shapely.geometry.polygon import Polygon
# from shapely.geometry.multipolygon import MultiPolygon
from explode import explode
from coord import coord
import matplotlib.pyplot as plt

In [20]:
#load data layers
f = '/Users/Zack/0_seawarden/python/detection_prep/targets_sar/'

#n = '2019_g25_100m_10-20_6m_02-01'
#n = '2019_g25_100m_10-20_1yr_02-01'
#n = '2018_g25_100m_10-20_6m_02-02'
n = '2018_g25_100m_10-20_1yr_02-02'

pts = gpd.read_file(f + n + '.geojson')

f2 = '/Users/Zack/0_greece/'
search_area = gpd.read_file(f2 + 'dem_eu/mask_search_area_100m_5km.shp')
exclusion_zones = gpd.read_file(f2 + 'AOI/exclusions_2020-01-07.geojson')
farm_sites = gpd.read_file(f2 + 'blue_bridge/fishcages_Greece_farm_polygons_2.shp')

#re-project layers
pts = pts.to_crs({'init': 'epsg:2100'})
search_area = search_area.to_crs({'init': 'epsg:2100'})
exclusion_zones = exclusion_zones.to_crs({'init': 'epsg:2100'})

print('detections', len(pts), pts.crs)
print('search area', len(search_area), search_area.crs)
print('exclusion zones', len(exclusion_zones), exclusion_zones.crs)

detections 10617 {'init': 'epsg:2100'}
search area 1 {'init': 'epsg:2100'}
exclusion zones 15 {'init': 'epsg:2100'}


In [21]:
# plt.rcParams['figure.figsize'] = (20, 10)
# ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
# pts.plot(markersize=5, facecolor='red', ax=ax)
# exclusion_zones.plot(ax=ax)

In [22]:
%%time 
#clip points to search area
clip = sjoin(pts, search_area, how='inner', op='within')
print('detections within search area:', len(clip))

detections within search area: 4540
Wall time: 2.43 s


In [23]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#clip.plot(markersize=5, facecolor = 'red', ax=ax)

In [24]:
%%time 
#combine all exclusion zones
exclusion_zones['Dissolve'] = 0
exclusion_zones_dis = exclusion_zones.dissolve(by='Dissolve')

Wall time: 0 ns


In [25]:
%%time 
#exclude points in exclusion areas
mask = ~clip.within(exclusion_zones_dis.loc[0, 'geometry'])
clip2 = clip.loc[mask]
print('detections outside of exclusion zones:', len(clip2))

detections outside of exclusion zones: 3525
Wall time: 265 ms


In [26]:
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#exclusion_zones_dis.plot(facecolor = 'blue', ax=ax)
#clip2.plot(markersize=5, facecolor = 'red', ax=ax)

In [27]:
%%time 
#buffer points, dissolve buffers to aggregate points near each other
buffer = gpd.GeoDataFrame(geometry = clip2.buffer(10))
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')

Wall time: 252 ms


In [28]:
%%time 
#explode polygon and generate centroids
buffer_exploded = explode(buffer_dis)      
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid)
print('detections after aggregation:', len(centroids))

detections after aggregation: 3507
Wall time: 1.72 s


In [29]:
%%time 
#buffer centroids and make square polygons
centroid_buffer = gpd.GeoDataFrame(geometry = centroids.buffer(50))
envelope = gpd.GeoDataFrame(geometry = centroid_buffer.envelope)

Wall time: 207 ms


In [30]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax=search_area.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroid_buffer.plot(markersize=5, facecolor = 'black', ax=ax)
#envelope.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [31]:
#plt.rcParams['figure.figsize'] = (20, 10)
#ax = centroids.loc[[0], 'geometry'].plot(markersize=10, facecolor = 'black')
#centroid_buffer.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)
#envelope.loc[[0], 'geometry'].plot(linewidth=.5, edgecolor = 'black', facecolor = 'none', ax=ax)

In [32]:
#set crs
envelope.crs = {'init' :'epsg:2100'}
envelope['geometry'] = envelope['geometry'].to_crs(epsg=4326)
print(envelope.crs)
envelope.head()

{'init': 'epsg:2100'}


,geometry
0,"POLYGON ((24.1117194014751 34.78588604855375, ..."
1,"POLYGON ((24.12551001425323 34.80271110336088,..."
2,"POLYGON ((24.11842575993763 34.84667187731439,..."
3,"POLYGON ((24.04262547882348 34.85034838207384,..."
4,"POLYGON ((24.03817278696284 34.85361676319124,..."


In [33]:
#extract lat/long for each square polygon (envelope)
coord_list = []
for i in range(len(envelope)):
    coords = mapping(envelope.geometry[i])['coordinates']
    coord_list.append(coords)

In [34]:
# #combine x/y point groups
coord_all = []
for i in range(len(coord_list)):
    coord_group = coord(coord_list[i]) #function to extract and format x/y points
    coord_group['id']=i
    coord_all.append(coord_group)
    
targets = pd.concat(coord_all)
print('total coordinates (5 per detection):', len(targets))
targets.head()

total coordinates (5 per detection): 17535


,x,y,id
0,24.111719401475103,34.78588604855375,0
1,24.112812427643906,34.78588504055781,0
2,24.11281365539641,34.78678681249847,0
3,24.111720617332125,34.78678782052806,0
4,24.111719401475103,34.78588604855375,0


In [35]:
#accuracy check
centroids_test = centroids.copy()
centroids_test.crs = {'init' :'epsg:2100'}
centroids_test['geometry'] = centroids_test['geometry'].to_crs(epsg=4326)

matches = centroids_test.intersects(farm_sites.unary_union)
count = centroids_test.loc[matches]

detections_n = len(centroids_test)
farm_n = len(farm_sites)
matches_n = len(count)

print('total detections:', len(pts))
print('detections in search area:', detections_n)
print('farm sites:', farm_n)
print('matches:', matches_n)
print('positive matches', round(matches_n / farm_n, 3))
print('overall accuracy', round(matches_n / detections_n, 3))

#plt.rcParams['figure.figsize'] = (20, 10)
#ax=farm_sites.plot(linewidth=.5, edgecolor = 'black', facecolor = 'none')
#centroids_test.plot(markersize=1, facecolor = 'red', ax=ax)

total detections: 10617
detections in search area: 3507
farm sites: 218
matches: 300
positive matches 1.376
overall accuracy 0.086


In [36]:
pts.to_file(f + n + '.shp')
targets.to_csv(f + n + '.csv', index = None, header=True)

centroids_test.crs = {'init' :'epsg:4326'}
centroids_test.to_file(f + n + '_v2.shp')
centroids_test.to_file(f + n + '_v2.geojson', driver='GeoJSON')